In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from statistics import mean 


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from statistics import mean 

# Load data
visit_df = pd.read_csv('Desktop/visit_forcasting_code.csv')
# Extract year and week number
visit_df['year'] = visit_df['WEEK'].astype(str).str[0:4]
visit_df['week_num'] = visit_df['WEEK'].astype(str).str[4:6]
visit_df.rename(columns={'WEEK': 'week_num','VISITORS':'visitors' ,'year': 'year', 'week_num': 'week'}, inplace=True)

visit_df['year'] = visit_df['year'].astype(int)
visit_df['week'] = visit_df['week'].astype(int)

# Calculate cumulative transactions and GMV
visit_df['cum_visits'] = visit_df.groupby(['year'])['visitors'].cumsum()

visit2022 = visit_df[visit_df['year'] == 2022]
visit2023 = visit_df[visit_df['year'] == 2023]

current_week = visit2023['week'].max()
x = 52 - current_week
visit2023=visit2023[visit2023['week'] < current_week]

pred_week_num = np.arange(current_week, 53)
pred_type = ['predicted'] * (x + 1)
pred_year = [2023] * (x + 1)

# Linear Regression Model
model = LinearRegression().fit(visit2023[['week']], visit2023['cum_visits'])
slope = model.coef_[0]
intercept = visit2023['cum_visits'].max()
pred_lm = np.arange(1, x + 2) * slope + intercept

lm_gmv = pd.DataFrame({'pred_year': pred_year, 'pred_week_num': pred_week_num, 'pred_lm': pred_lm, 'pred_type': pred_type})
lm_gmv['cum_lm_visits'] = lm_gmv['pred_lm'].cumsum()

# Year-over-Year comparison

growth_from_2023 = visit2022[visit2022['week'] > current_week-1]['visitors'].cumsum()
pred_visits = growth_from_2023 + visit2023['cum_visits'].max()

short_2022=visit2022[visit2022['week'] < current_week]
new_2022 = short_2022[['week', 'visitors']].copy()

##Finding the difference between the average weekly gmv between last year and this year. Numbers below 
#1 mean that this year is doing
##Better than last year i.e. if we get .7 then we have currently ~70% less than last year at this time, 
#and could expect our WOW model to do 70% worse, so we multiply the WOW growth from 2023 by this value 


YOY_comparison=mean(visit2023['visitors'].values)/mean(new_2022['visitors'].values)
scaled_pred_visits = growth_from_2023 * YOY_comparison + visit2023['cum_visits'].max()

final_df=pd.DataFrame({'fy_year': pred_year, 'fy_week': pred_week_num,'wow_growth_model': pred_visits, 'scaled_wow_growth_model': scaled_pred_visits,'linear_model': pred_lm})


In [11]:
final_df

,year,week,wow_growth_model,scaled_wow_growth_model,linear_model
38,2023,39,24368742,2.420599e+07,2.415202e+07
39,2023,40,25128764,2.481714e+07,2.476620e+07
40,2023,41,25868933,2.541233e+07,2.538038e+07
41,2023,42,26574411,2.597962e+07,2.599457e+07
42,2023,43,27309604,2.657081e+07,2.660875e+07
43,2023,44,27966120,2.709873e+07,2.722293e+07
44,2023,45,28537214,2.755796e+07,2.783711e+07
45,2023,46,29141566,2.804393e+07,2.845130e+07
46,2023,47,29763348,2.854392e+07,2.906548e+07
47,2023,48,30407795,2.906214e+07,2.967966e+07
